In [1]:
import re
from pprint import pprint

import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [16]:
data_path = "uri_scores.csv"
file = pd.read_csv(data_path)

def iter_urls(file):
    for i, row in file.iterrows():
        url, text = row["orig_url"], str(row["text"])
        yield url, text


class TaggedWebpageDocument(object):
    def __iter__(self):
        for url, text in iter_urls(file):
            words = [c for c in re.split(r"\s+", re.sub(r"[^\w\s]+", " ", text)) if len(c) > 0]
            yield TaggedDocument(words, [url])


documents = TaggedWebpageDocument()
model = Doc2Vec(documents, window=7, vector_size=256, workers=4)
model.save("oos-doc2vec")

/home/rolv-arild/PycharmProjects/soc/venv/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
model = Doc2Vec.load("oos-doc2vec")

In [7]:
pprint(model.docvecs.most_similar(positive=["https://itunes.apple.com"], topn=10))
pprint(model.docvecs.most_similar(positive=["https://begrep.difi.no"], topn=10))
pprint(model.docvecs.most_similar(positive=["https://www.sharp.fi"], topn=10))

[('http://itunes.apple.com', 0.9400783777236938),
 ('http://ax.itunes.apple.com', 0.9343364238739014),
 ('https://geo.itunes.apple.com', 0.8924614787101746),
 ('http://www.apple.com', 0.8770399689674377),
 ('http://apple.com', 0.8673635125160217),
 ('https://www.apple.com', 0.8480823040008545),
 ('http://livepage.apple.com', 0.8468115329742432),
 ('http://store.apple.com', 0.8216663002967834),
 ('https://maps.apple.com', 0.8066335916519165),
 ('https://amzn.to', 0.7956060171127319)]
[('http://kopweb.hig.no', 0.7531193494796753),
 ('http://www.arabiskfilm.no', 0.7507939338684082),
 ('http://www.haukeboe-computing.no', 0.7448505163192749),
 ('http://heatsec.datasenter.no', 0.7444443106651306),
 ('http://www.solberg.as', 0.7441533803939819),
 ('https://uks.hypersys.no', 0.74281245470047),
 ('http://teddy-jenssen.datasenter.no', 0.7424317002296448),
 ('https://scholar.google.no', 0.7416311502456665),
 ('http://proxy.helsebiblioteket.no', 0.7394607067108154),
 ('http://www.hig.no', 0.736959

In [17]:
print(file.shape, len(model.docvecs.doctags))
fw = open("oos-doc2vec.csv", "w")
fw.write(",".join(file.columns) + "," + ",".join([f"v{i}" for i in range(256)]) + "\n")
for tag in model.docvecs.doctags:
    vec = model.docvecs[tag]
    rest = file[file["orig_url"] == tag].values[0]
    fw.write(",".join([str(v).replace(",", "|") for v in rest]) + "," + ",".join([str(v) for v in vec]) + "\n")


(7336, 40) 7334
